In [9]:
!pip install -U scikit-learn

     ---------------------------------------- 7.2/7.2 MB 8.5 MB/s eta 0:00:00
     ------------------------------------- 307.0/307.0 KB 19.8 MB/s eta 0:00:00


In [11]:
!pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import shutil
import json
import re
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.utils.data as data_utils
from torch.nn.modules import MSELoss, L1Loss

from sklearn.preprocessing import MultiLabelBinarizer
import glob
import csv
import cv2
import random
from PIL import Image
from itertools import product

In [2]:
path1 = "./Movie_Poster_Metadata/groundtruth"
temp_path = "./Movie_Poster_Metadata/temp_groundtruth"
path2 = "./Movie_Poster_Metadata/updated_groundtruth"

### Reading the input file and creating a clean one

In [13]:
dir_list = os.listdir(path1)
 
if not os.path.exists(temp_path):
  os.makedirs(temp_path)    

if not os.path.exists(path2):
  os.makedirs(path2)



for file_name in dir_list:
    
    with open(path1+'/'+file_name,'r',encoding='utf-16-le') as file1:

        temp_file = open(temp_path+'/'+file_name,'w',encoding='utf-8')

        for line in file1.readlines():

            line = line.replace("}\n","},\n")
            
            # reading all lines that begin with "  "_id""
            y = re.findall("^  \"_id\"", line)
            if not y:
                temp_file.write(line)

    file1.close()
    temp_file.close()

In [14]:
dir_list = os.listdir(temp_path)
 
for file_name in dir_list:
    
    with open(temp_path+'/'+file_name,'r',encoding='utf-8') as temp_file:
    
        file2 = open(path2+'/'+file_name,'w',encoding='utf-8')

        lines = temp_file.readlines()
        lines = lines[1:-1]

        file2.write("[{")
        file2.writelines(lines)
        file2.write("}]")
        
    temp_file.close()
    file2.close()

shutil.rmtree(temp_path)  

### Function to append all the json objects into dataframe 

In [6]:
dir_list = os.listdir(path2)

movies_df = pd.DataFrame()

for file_name in dir_list:    

#     try:
    df = pd.read_json(path2+'/'+file_name,encoding='utf-8',orient='records')
    df = df[['imdbID','Director','Genre','imdbRating']]
    movies_df = pd.concat([movies_df,df], ignore_index=True)

#     except:
#         print(file_name)
        
print(movies_df.dtypes)
print(movies_df.head(20))
print(movies_df.shape)

imdbID        object
Director      object
Genre         object
imdbRating    object
dtype: object
       imdbID                                  Director  \
0   tt0080684                            Irvin Kershner   
1   tt0081562                            Sidney Poitier   
2   tt0080339  Jim Abrahams, David Zucker, Jerry Zucker   
3   tt0080377                            Buddy Van Horn   
4   tt0081375                              Howard Zieff   
5   tt0080549                             Michael Apted   
6   tt0081529                               Hal Needham   
7   tt0080453                            Randal Kleiser   
8   tt0080455                               John Landis   
9   tt0081283                            Robert Redford   
10  tt0081353                             Robert Altman   
11  tt0081696                             James Bridges   
12  tt0081505                           Stanley Kubrick   
13  tt0081480                              Jay Sandrich   
14  tt0080520    

### Create multi-hot encoded genre vectors

In [55]:
#split genres in movies_df
genres_df = movies_df['Genre'].str.split(', ', expand=True)
genres_df.columns = ['Genre'+str(i) for i in genres_df.columns]
movies_expanded_df = pd.concat([movies_df,genres_df], axis=1)

#print(movies_expanded_df.head(10))

#get unique genres
unique_genres = (((movies_expanded_df["Genre0"].append(movies_expanded_df["Genre1"])).append(movies_expanded_df["Genre1"])).append(movies_expanded_df["Genre1"])).unique()
print(unique_genres)
#print(np.where(unique_genres == "Biography")[0][0])

#replace genre strings with ids
for genre in unique_genres:
    if genre != None:
        replacement = np.where(unique_genres == genre)[0][0]
        
        movies_expanded_df["Genre0"].replace(genre, str(replacement), inplace=True)
        movies_expanded_df["Genre1"].replace(genre, str(replacement), inplace=True)
        movies_expanded_df["Genre2"].replace(genre, str(replacement), inplace=True)
        movies_expanded_df["Genre3"].replace(genre, str(replacement), inplace=True)
print(movies_expanded_df.head(10))

['Action' 'Comedy' 'Biography' 'Adventure' 'Drama' 'Mystery' 'Fantasy'
 'Crime' 'Horror' 'Music' 'Western' 'Documentary' 'Family' 'Animation'
 'Romance' 'Musical' 'Sci-Fi' 'Thriller' 'Short' 'War' 'Adult'
 'Reality-TV' 'N/A' 'Game-Show' 'History' 'News' 'Sport' 'Talk-Show' None]
      imdbID                                  Director  \
0  tt0080684                            Irvin Kershner   
1  tt0081562                            Sidney Poitier   
2  tt0080339  Jim Abrahams, David Zucker, Jerry Zucker   
3  tt0080377                            Buddy Van Horn   
4  tt0081375                              Howard Zieff   
5  tt0080549                             Michael Apted   
6  tt0081529                               Hal Needham   
7  tt0080453                            Randal Kleiser   
8  tt0080455                               John Landis   
9  tt0081283                            Robert Redford   

                        Genre imdbRating Genre0 Genre1 Genre2 Genre3  
0  Action,

C:\Users\carol\AppData\Local\Temp\ipykernel_84916\3553992389.py:9: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unique_genres = (((movies_expanded_df["Genre0"].append(movies_expanded_df["Genre1"])).append(movies_expanded_df["Genre1"])).append(movies_expanded_df["Genre1"])).unique()
C:\Users\carol\AppData\Local\Temp\ipykernel_84916\3553992389.py:9: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unique_genres = (((movies_expanded_df["Genre0"].append(movies_expanded_df["Genre1"])).append(movies_expanded_df["Genre1"])).append(movies_expanded_df["Genre1"])).unique()


In [60]:
mlb = MultiLabelBinarizer()
mlb.fit(movies_expanded_df["Genre0"])

new_col_names = ["ID_%s" % c for c in mlb.classes_]

# Create new DataFrame with transformed/one-hot encoded IDs
ids = pd.DataFrame(mlb.fit_transform(movies_expanded_df["Genre0"]), columns=new_col_names)

# Concat with original "Genre" column
genres_df = pd.concat( [movies_expanded_df, ids], axis=1 )

print(genres_df.head(10))

      imdbID                                  Director  \
0  tt0080684                            Irvin Kershner   
1  tt0081562                            Sidney Poitier   
2  tt0080339  Jim Abrahams, David Zucker, Jerry Zucker   
3  tt0080377                            Buddy Van Horn   
4  tt0081375                              Howard Zieff   
5  tt0080549                             Michael Apted   
6  tt0081529                               Hal Needham   
7  tt0080453                            Randal Kleiser   
8  tt0080455                               John Landis   
9  tt0081283                            Robert Redford   

                        Genre imdbRating Genre0 Genre1 Genre2 Genre3  ID_0  \
0  Action, Adventure, Fantasy        8.8      0      3      6   None     1   
1               Comedy, Crime        6.8      1      7   None   None     0   
2                      Comedy        7.8      1   None   None   None     0   
3              Action, Comedy        6.0      0  

### Augmenting the data set

In [5]:
path3 = "./Movie_Poster_Dataset"

# Going through all jpg-files, they are chopped up into 100x100 chunks and saved into a new folder
for dirname in os.listdir(path3):
    for filename in os.listdir(path3 + "/" + dirname):
        name, ext = os.path.splitext(filename)
        if(ext == '.jpg'):
            image = Image.open(os.path.join(path3 + "/" + dirname, filename))
            width, height = image.size
            chopsize = 100
            for x0 in range(0, width, chopsize):
                for y0 in range(0, height, chopsize):
                    if(y0+chopsize <= height and x0+chopsize <= width):
                        box = (x0, y0, x0+chopsize, y0+chopsize)
                        image.crop(box).save('./Movie_Poster_Dataset_Cropped/%s.x%03d.y%03d.jpg' % (filename.replace('.jpg',''), x0, y0))


### Passing the images through a convolutional network

In [61]:
#training controls
batch_size = 20
epochs = 2
training_size = 0.7
learning_rate = 0.001
dropout = [0.3, 0.3, 0.3, 0.3, 0.2, 0.2, 0.2, 0.2, 0.15]
# input image dimensions
img_rows, img_cols = 100, 100

In [64]:
# the data holders
x_test = []
x_train = []
y_test= []
y_train= []

#the list of image files in SampleMoviePosters folder
flist=glob.glob('./Movie_Poster_Dataset_Cropped/*.jpg')

#setting the length of training data
length=int(len(flist)*training_size)

#currently using 1000 images
length=1000*training_size

#extracting the data about the images that are available
i=0
for filename in flist:
    
    imdb_id = filename[0:filename.index(".")]
            
    img = np.array(cv2.imread(filename))
    img = np.swapaxes(img, 2,0)
    img = np.swapaxes(img, 2,1)

    if(i<length):
        x_train.append(img)
        
        #generating a random genre array (desired output)
        genre_arr = np.empty([28])
    
        for x in range(28):
            genre_arr[x] = 0
        
        #idx = (genres_df.iloc[imdb_id])[Genre0]
        rnd = random.randint(0, 27)
        genre_arr[rnd] = 1
        y_train.append(genre_arr)

    else:
        x_test.append(img)
        
        #generating a random genre array (desired output)
        genre_arr = np.empty([28])
    
        for x in range(28):
            genre_arr[x] = 0
        
        #idx = (genres_df.iloc[imdb_id])[Genre0]
        rnd = random.randint(0, 27)
        genre_arr[rnd] = 1
        y_test.append(genre_arr)
    i+=1
    #stopping after 1000 images
    if i==1000:
        break

In [66]:
#converting the data from lists to numpy arrays
x_train=np.asarray(x_train,dtype=float)
x_test=np.asarray(x_test,dtype=float)
y_train=np.asarray(y_train,dtype=float)
y_test=np.asarray(y_test,dtype=float)

#scaling down the RGB data
x_train /= 255
x_test /= 255

#printing stats about the features
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

train_length = x_train.shape[0]

x_train=torch.from_numpy(x_train)
x_test=torch.from_numpy(x_test)
y_train=torch.from_numpy(y_train)
y_test=torch.from_numpy(y_test)

train = data_utils.TensorDataset(x_train, y_train)
train_loader = data_utils.DataLoader(train, batch_size=batch_size, shuffle=True)

test = data_utils.TensorDataset(x_test, y_test)
test_loader = data_utils.DataLoader(test, batch_size=batch_size, shuffle=False)

x_train shape: (700, 3, 100, 100)
700 train samples
300 test samples


In [68]:
class Net(nn.Module):
    def __init__(self, input_shape=(3, img_rows, img_cols)):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 128, kernel_size=2)
        self.conv1_drop = nn.Dropout2d(p=dropout[0])
        self.conv2 = nn.Conv2d(128, 64, kernel_size=2)
        self.conv2_drop = nn.Dropout2d(p=dropout[1])
        self.conv3 = nn.Conv2d(64, 64, kernel_size=2)
        self.conv3_drop = nn.Dropout2d(p=dropout[2])
        self.conv4 = nn.Conv2d(64, 64, kernel_size=2)
        self.conv4_drop = nn.Dropout2d(p=dropout[3])
        self.conv5 = nn.Conv2d(64, 32, kernel_size=2)
        self.conv5_drop = nn.Dropout2d(p=dropout[4])
        self.conv6 = nn.Conv2d(32, 16, kernel_size=2)
        self.conv6_drop = nn.Dropout2d(p=dropout[5])
        
        n_size = self._get_conv_output(input_shape)
        
        self.fc1 = nn.Linear(n_size, 16)
        self.fc1_drop = nn.Dropout(p=dropout[6])
        self.fc2 = nn.Linear(16, 16)
        self.fc2_drop = nn.Dropout(p=dropout[7])
        self.fc3 = nn.Linear(16, 8)
        self.fc3_drop = nn.Dropout(p=dropout[8])
        self.fc4 = nn.Linear(8, 1)
        
    def _get_conv_output(self, shape):
        bs = 1
        input = Variable(torch.rand(bs, *shape))
        output_feat = self._forward_features(input)
        n_size = output_feat.data.view(bs, -1).size(1)
        return n_size
        
    def _forward_features(self, x):
        x = F.relu(F.max_pool2d(self.conv1_drop(self.conv1(x)), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = F.relu(F.max_pool2d(self.conv3_drop(self.conv3(x)), 2))
        x = F.relu(F.max_pool2d(self.conv4_drop(self.conv4(x)), 2))
        x = F.relu(F.max_pool2d(self.conv5_drop(self.conv5(x)), 2))
        #x = F.relu(F.max_pool2d(self.conv6_drop(self.conv6(x)), 2))
        return x
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1_drop(self.conv1(x)), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = F.relu(F.max_pool2d(self.conv3_drop(self.conv3(x)), 2))
        x = F.relu(F.max_pool2d(self.conv4_drop(self.conv4(x)), 2))
        x = F.relu(F.max_pool2d(self.conv5_drop(self.conv5(x)), 2))
        #x = F.relu(F.max_pool2d(self.conv6_drop(self.conv6(x)), 2))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1_drop(self.fc1(x)))
        x = F.relu(self.fc2_drop(self.fc2(x)))
        x = F.relu(self.fc3_drop(self.fc3(x)))
        x = self.fc4(x)
        return x

model = Net()
criterion = MSELoss(size_average=True)
human_criterion = L1Loss(size_average=True)
optimizer = optim.RMSprop(model.parameters(), lr=learning_rate,
            alpha=0.9, eps=1e-08, weight_decay=0.0)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        human_loss= human_criterion(output, target)
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data, human_loss.data))

def test():
    print('test')
    model.eval()
    test_loss = 0
    correct = 0
    human_loss = 0
    i = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        data, target = Variable(data, volatile=True).float(), Variable(target).float()
        output = model(data)
        loss = criterion(output, target)
        test_loss += loss
        human_loss += human_criterion(output, target)
        if loss==0:
            correct+=1

    print('\nTest set: \nAverage sq_loss: {:.4f} \nAverage abs_loss: {:.4f} \nGuessed 100% correct: {:.4f}\n'.format(test_loss.data/i, human_loss.data/i, correct))

model.float()
print(model)
for epoch in range(0, epochs):
    train(epoch)
    test()

Net(
  (conv1): Conv2d(3, 128, kernel_size=(2, 2), stride=(1, 1))
  (conv1_drop): Dropout2d(p=0.3, inplace=False)
  (conv2): Conv2d(128, 64, kernel_size=(2, 2), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.3, inplace=False)
  (conv3): Conv2d(64, 64, kernel_size=(2, 2), stride=(1, 1))
  (conv3_drop): Dropout2d(p=0.3, inplace=False)
  (conv4): Conv2d(64, 64, kernel_size=(2, 2), stride=(1, 1))
  (conv4_drop): Dropout2d(p=0.3, inplace=False)
  (conv5): Conv2d(64, 32, kernel_size=(2, 2), stride=(1, 1))
  (conv5_drop): Dropout2d(p=0.2, inplace=False)
  (conv6): Conv2d(32, 16, kernel_size=(2, 2), stride=(1, 1))
  (conv6_drop): Dropout2d(p=0.2, inplace=False)
  (fc1): Linear(in_features=128, out_features=16, bias=True)
  (fc1_drop): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (fc2_drop): Dropout(p=0.2, inplace=False)
  (fc3): Linear(in_features=16, out_features=8, bias=True)
  (fc3_drop): Dropout(p=0.15, inplace=False)
  (fc4): Linear(in_features

C:\Users\carol\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:529: UserWarning: Using a target size (torch.Size([20, 28])) that is different to the input size (torch.Size([20, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\carol\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:96: UserWarning: Using a target size (torch.Size([20, 28])) that is different to the input size (torch.Size([20, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Train Epoch: 0 [0/700 (0%)]	Loss: 0.040060 0.074887
Train Epoch: 0 [20/700 (3%)]	Loss: 0.039105 0.068092
Train Epoch: 0 [40/700 (6%)]	Loss: 0.037542 0.057164
Train Epoch: 0 [60/700 (9%)]	Loss: 0.038051 0.075392
Train Epoch: 0 [80/700 (11%)]	Loss: 0.036772 0.049336
Train Epoch: 0 [100/700 (14%)]	Loss: 0.035193 0.054444
Train Epoch: 0 [120/700 (17%)]	Loss: 0.034795 0.065989
Train Epoch: 0 [140/700 (20%)]	Loss: 0.035384 0.065460
Train Epoch: 0 [160/700 (23%)]	Loss: 0.035779 0.057114
Train Epoch: 0 [180/700 (26%)]	Loss: 0.035289 0.058904
Train Epoch: 0 [200/700 (29%)]	Loss: 0.035361 0.058584
Train Epoch: 0 [220/700 (31%)]	Loss: 0.035739 0.063090
Train Epoch: 0 [240/700 (34%)]	Loss: 0.035427 0.057253
Train Epoch: 0 [260/700 (37%)]	Loss: 0.035181 0.063147
Train Epoch: 0 [280/700 (40%)]	Loss: 0.035759 0.064822
Train Epoch: 0 [300/700 (43%)]	Loss: 0.036004 0.059184
Train Epoch: 0 [320/700 (46%)]	Loss: 0.034960 0.059980
Train Epoch: 0 [340/700 (49%)]	Loss: 0.035042 0.059134
Train Epoch: 0 [360/

C:\Users\carol\AppData\Local\Temp\ipykernel_84916\559374431.py:86: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True).float(), Variable(target).float()



Test set: 
Average sq_loss: 0.0344 
Average abs_loss: 0.0672 
Guessed 100% correct: 0.0000

Train Epoch: 1 [0/700 (0%)]	Loss: 0.034965 0.064767
Train Epoch: 1 [20/700 (3%)]	Loss: 0.034955 0.069272
Train Epoch: 1 [40/700 (6%)]	Loss: 0.034777 0.064887
Train Epoch: 1 [60/700 (9%)]	Loss: 0.034708 0.070597
Train Epoch: 1 [80/700 (11%)]	Loss: 0.034495 0.070145
Train Epoch: 1 [100/700 (14%)]	Loss: 0.034734 0.064066
Train Epoch: 1 [120/700 (17%)]	Loss: 0.034731 0.064574
Train Epoch: 1 [140/700 (20%)]	Loss: 0.034830 0.064226
Train Epoch: 1 [160/700 (23%)]	Loss: 0.034589 0.069115
Train Epoch: 1 [180/700 (26%)]	Loss: 0.034868 0.062671
Train Epoch: 1 [200/700 (29%)]	Loss: 0.034700 0.064548
Train Epoch: 1 [220/700 (31%)]	Loss: 0.034725 0.069050
Train Epoch: 1 [240/700 (34%)]	Loss: 0.034626 0.067209
Train Epoch: 1 [260/700 (37%)]	Loss: 0.034780 0.064893
Train Epoch: 1 [280/700 (40%)]	Loss: 0.034563 0.067498
Train Epoch: 1 [300/700 (43%)]	Loss: 0.034598 0.064039
Train Epoch: 1 [320/700 (46%)]	Loss: 

### Passing the images through object detection

In [3]:
batch_size = 20
epochs = 2
training_size = 0.7
learning_rate = 0.001
img_rows, img_cols = 100, 100

x_test_obj = []
x_train_obj = []
y_test_obj= []
y_train_obj= []
tempY_obj = []


In [4]:
path = "./Movie_Poster_Dataset_Cropped"

#pretrained YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

counter = 1000

for filename in os.listdir(path):
    name, ext = os.path.splitext(filename)
    if(ext == '.jpg'):
        image = Image.open(os.path.join(path, filename))
        results = model(image, size = 100)
            
        #create an array for the 91 object categories and set initial confidence to 0
        obj_arr = np.empty([91])
        for x in range(91):
            obj_arr[x] = 0.0

        #update the confidence values according to the object detection results
        for obj in results.pandas().xyxy[0]:
            index =  results.pandas().xyxy[0]['class']
            obj_arr[index] = obj_arr[index] + results.pandas().xyxy[0].confidence
        
        x_train_obj.append(obj_arr)
        
        #generating a random genre array (desired output)
        genre_arr = np.empty([28])
    
        for x in range(28):
            genre_arr[x] = 0
        
        rnd = random.randint(0, 27)
        genre_arr[rnd] = 1
        y_train_obj.append(genre_arr)
        
        counter -= 1
        if counter == 0:
            break

Using cache found in C:\Users\carol/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-5-5 torch 1.11.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [5]:
x_train_obj = np.asarray(x_train_obj, dtype=float)
y_train_obj = np.asarray(y_train_obj, dtype=float)

#printing stats about the features
print('x_train shape:', x_train_obj.shape)
print(x_train_obj.shape[0], 'train samples')

train_length = x_train_obj.shape[0]

x_train shape: (1000, 91)
1000 train samples


In [7]:
x_train_obj= torch.from_numpy(x_train_obj)
y_train_obj=torch.from_numpy(y_train_obj)

train = data_utils.TensorDataset(x_train_obj, y_train_obj)
train_loader = data_utils.DataLoader(train, batch_size=batch_size, shuffle=True)

In [8]:
#fully connected layer after object detection
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        #fully connected layer
        self.fc1 = nn.Linear(91, 28)
        
    def forward(self, x):
        x = self.fc1(x)
        output = F.log_softmax(x, dim=1)
        return output

model = Net()

result = model.train()
criterion = MSELoss(size_average=True)
human_criterion = L1Loss(size_average=True)
optimizer = optim.RMSprop(model.parameters(), lr=learning_rate,
            alpha=0.9, eps=1e-08, weight_decay=0.0)


C:\Users\carol\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
